In [1]:
# BLOQUE 0: base común (importaciones + tipos para OOP)
from dataclasses import dataclass
from typing import Callable, List

@dataclass
class RootResult:
    """Resultado estándar para todos los métodos de raíces."""
    root: float
    f_at_root: float
    iterations: int
    converged: bool
    history: List[float]

class RootFinder:
    """Clase base simple con tolerancia y límite de iteraciones."""
    def __init__(self, tol: float = 1e-8, max_iter: int = 100):
        self.tol = tol
        self.max_iter = max_iter
    def solve(self) -> RootResult:
        raise NotImplementedError("Implementar en subclases.")


In [2]:
# BLOQUE 1: BISECCIÓN (función + clase OOP)

def biseccion(f, a, b, tol=1e-8, max_iter=100):
    fa, fb = f(a), f(b)
    if fa * fb > 0:
        raise ValueError("Bisección: f(a)*f(b) debe ser < 0 (cambio de signo).")
    for i in range(max_iter):
        m = (a + b) / 2
        fm = f(m)
        if abs(fm) < tol or abs(b - a) < tol:
            return m, i + 1
        if fa * fm < 0:
            b, fb = m, fm
        else:
            a, fa = m, fm
    return m, max_iter

class Bisection(RootFinder):
    """Implementación OOP: envuelve la función 'biseccion'."""
    def __init__(self, f: Callable[[float], float], a: float, b: float, tol: float = 1e-8, max_iter: int = 100):
        super().__init__(tol, max_iter); self.f, self.a, self.b = f, a, b
    def solve(self) -> RootResult:
        root, it = biseccion(self.f, self.a, self.b, tol=self.tol, max_iter=self.max_iter)
        return RootResult(root, self.f(root), it, True, [])


In [4]:
#Falsa posición
class FixedPoint(RootFinder):
    """
    Requiere reescribir f(x)=0 como x=g(x). Converge si |g'(x*)|<1 cerca de la raíz.
    """
    def __init__(self, g: Callable[[float], float], x0: float, tol: float = 1e-8, max_iter: int = 100):
        super().__init__(tol, max_iter); self.g, self.x0 = g, x0
    def solve(self) -> RootResult:
        x = self.x0
        hist: List[float] = [x]
        for k in range(1, self.max_iter + 1):
            x_new = self.g(x); hist.append(x_new)
            if abs(x_new - x) < self.tol:
                return RootResult(x_new, x_new - self.g(x_new), k, True, hist)  # f≈x-g(x)
            x = x_new
        return RootResult(x, x - self.g(x), self.max_iter, False, hist)


In [5]:
#NEWRTON
def newton_raphson(f, df, x0, tol=1e-8, max_iter=100):
    x = x0
    for i in range(max_iter):
        fx, dfx = f(x), df(x)
        if dfx == 0:
            raise ZeroDivisionError("Newton: derivada nula.")
        x_new = x - fx / dfx
        if abs(x_new - x) < tol or abs(fx) < tol:
            return x_new, i + 1
        x = x_new
    return x, max_iter

class NewtonRaphson(RootFinder):
    def __init__(self, f: Callable[[float], float], df: Callable[[float], float], x0: float, tol: float = 1e-8, max_iter: int = 100):
        super().__init__(tol, max_iter); self.f, self.df, self.x0 = f, df, x0
    def solve(self) -> RootResult:
        x = self.x0
        hist: List[float] = [x]
        for k in range(1, self.max_iter + 1):
            fx, dfx = self.f(x), self.df(x)
            if dfx == 0:
                return RootResult(x, fx, k-1, False, hist)
            x_new = x - fx / dfx
            hist.append(x_new)
            if abs(x_new - x) < self.tol or abs(fx) < self.tol:
                return RootResult(x_new, self.f(x_new), k, True, hist)
            x = x_new
        return RootResult(x, self.f(x), self.max_iter, False, hist)


In [6]:
#SECANTE
class Secant(RootFinder):
    """No requiere derivada; usa dos semillas x0 y x1."""
    def __init__(self, f: Callable[[float], float], x0: float, x1: float, tol: float = 1e-8, max_iter: int = 100):
        super().__init__(tol, max_iter); self.f, self.x0, self.x1 = f, x0, x1
    def solve(self) -> RootResult:
        f = self.f
        x0, x1 = self.x0, self.x1
        f0, f1 = f(x0), f(x1)
        hist: List[float] = [x0, x1]
        for k in range(1, self.max_iter + 1):
            denom = (f1 - f0)
            if denom == 0:
                return RootResult(x1, f1, k-1, False, hist)
            x2 = x1 - f1 * (x1 - x0) / denom
            hist.append(x2)
            if abs(x2 - x1) < self.tol or abs(f(x2)) < self.tol:
                return RootResult(x2, f(x2), k, True, hist)
            x0, x1 = x1, x2
            f0, f1 = f1, f(x1)
        return RootResult(x1, f1, self.max_iter, False, hist)


In [7]:
#BRENT
class Brent(RootFinder):
    """
    Requiere intervalo [a,b] con f(a)*f(b) < 0.
    Combina bisección, secante e interpolación cuadrática inversa.
    """
    def __init__(self, f: Callable[[float], float], a: float, b: float, tol: float = 1e-8, max_iter: int = 100):
        super().__init__(tol, max_iter); self.f, self.a, self.b = f, a, b
    def solve(self) -> RootResult:
        f, a, b = self.f, self.a, self.b
        fa, fb = f(a), f(b)
        if fa == 0: return RootResult(a, 0.0, 0, True, [a])
        if fb == 0: return RootResult(b, 0.0, 0, True, [b])
        if fa * fb > 0: raise ValueError("Brent: f(a)*f(b) debe ser < 0.")
        c, fc = a, fa
        d = e = b - a
        hist: List[float] = [a, b]
        for k in range(1, self.max_iter + 1):
            if fb * fc > 0:
                c, fc = a, fa
                d = e = b - a
            if abs(fc) < abs(fb):
                a, b, c = b, c, b
                fa, fb, fc = fb, fc, fb
            tol_act = 2.0 * self.tol * max(1.0, abs(b))
            m = 0.5 * (c - b)
            if abs(m) <= tol_act or fb == 0.0:
                return RootResult(b, fb, k, True, hist)
            if abs(e) >= tol_act and abs(fa) > abs(fb):
                s = fb / fa
                if a == c:
                    p = 2.0 * m * s; q = 1.0 - s
                else:
                    q = fa / fc; r = fb / fc
                    p = s * (2.0 * m * q * (q - r) - (b - a) * (r - 1.0))
                    q = (q - 1.0) * (r - 1.0) * (s - 1.0)
                if p > 0: q = -q
                p = abs(p)
                if 2.0 * p < min(3.0 * m * q - abs(tol_act * q), abs(e * q)):
                    e, d = d, p / q
                else:
                    d = m; e = m
            else:
                d = m; e = m
            a, fa = b, fb
            if abs(d) > tol_act:
                b = b + d if d > 0 else b - abs(d)
            else:
                b = b + tol_act if m > 0 else b - tol_act
            fb = f(b)
            hist.append(b)
        return RootResult(b, fb, self.max_iter, False, hist)


In [11]:
# BLOQUE 7: DEMO mínima (puedes comentar/editar a gusto)
def f(x): return x**3 - x - 2
def df(x): return 3*x**2 - 1
g = lambda x: (x + 2)**(1/3)

ejemplos = {
    "Bisección":      Bisection(f, 1, 2).solve(),
    "Falsa Posición": FalsePosition(f, 1, 2).solve(),
    "Newton":         NewtonRaphson(f, df, 1.5).solve(),
    "Secante":        Secant(f, 1.0, 2.0).solve(),
    "Punto Fijo":     FixedPoint(g, 1.0, tol=1e-10, max_iter=200).solve(),
    "Brent":          Brent(f, 1, 2).solve(),
}
for name, r in ejemplos.items():
    print(f"{name:14s} → raíz={r.root:.10f} | f(root)={r.f_at_root: .2e} | it={r.iterations:2d} | ok={r.converged}")

#Otro ejemplo por si qieren proba en la demo
#def f(x): return x**3 - 2*x**2 - 5
#def df(x): return 3*x**2 - 4*x
#g = lambda x: (2*x**2 + 5)**(1/3)

#a, b = 2, 3
#x0 = 2.5
#x0_fp = 2.0


Bisección      → raíz=1.5213797055 | f(root)=-7.64e-09 | it=28 | ok=True
Falsa Posición → raíz=1.5213797063 | f(root)=-2.93e-09 | it=17 | ok=True
Newton         → raíz=1.5213797068 | f(root)= 0.00e+00 | it= 4 | ok=True
Secante        → raíz=1.5213797080 | f(root)= 7.02e-09 | it= 6 | ok=True
Punto Fijo     → raíz=1.5213797068 | f(root)=-5.44e-12 | it=13 | ok=True
Brent          → raíz=1.5213797067 | f(root)=-4.90e-10 | it= 8 | ok=True
